# Загрузка данных

In [466]:
import pandas as pd
import json

In [467]:
FILE_NAME='data10-04.xlsx'

In [468]:
df = pd.read_excel(f'datasets/{FILE_NAME}')

In [469]:
df.head()

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created
0,64b18dc9-69d0-4286-887f-7469ebc1b47d,9770f581-b89f-4fb3-94f5-93173bb85ad1,ED_Meter_Readings-ListView,1,"{""LINK"":""50fb47a1-6711-4e65-9232-3700daaa58bd""...",2023-04-09 16:42:47.637
1,64b18dc9-69d0-4286-887f-7469ebc1b47d,5f97a8be-b3ad-4982-9191-fbed301b3c69,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 16:42:47.980
2,64b18dc9-69d0-4286-887f-7469ebc1b47d,d4c614dd-3114-4326-8ddd-bb9d74382b03,DD_Docs_Out-ListView,0,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:48.627
3,64b18dc9-69d0-4286-887f-7469ebc1b47d,9ee02cde-4a6f-4c51-8e3b-b14ddae38bee,DD_Files-ListView,0,"{""LINK"":""e010d6d0-1df5-790d-19fe-b007d7be4207""...",2023-04-09 16:42:49.557
4,64b18dc9-69d0-4286-887f-7469ebc1b47d,6829a2cd-007c-45d8-88ca-e143617555c4,DD_Docs-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:50.010


In [470]:
df['D_Date_Created'] = pd.to_datetime(df['D_Date_Created'])
df['N_Operation'] = df['N_Operation'].astype('int8')

In [471]:
df.drop_duplicates(inplace=True)

In [472]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29561 entries, 0 to 31944
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   F_User          29561 non-null  object        
 1   C_Record_ID     29561 non-null  object        
 2   C_Table         29561 non-null  object        
 3   N_Operation     29561 non-null  int8          
 4   C_Json          29216 non-null  object        
 5   D_Date_Created  29561 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int8(1), object(4)
memory usage: 1.4+ MB


# Алгоритмы

In [473]:
users = df['F_User'].unique()

In [474]:
use_records = []
correct_records = []

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [475]:
dict_algoritm_1 = {}
for i in range(len(users)):
    dict_algoritm_1[users[i]] = []

In [476]:
def algoritm_1(user_id):    
    items = df.loc[df['F_User'] == user_id].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'ED_Meter_Readings-ListView': [],
        'DD_Doc_Details-ListView': [],
        'DD_Docs_Out-ListView': [],
        'DD_Files-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'ED_Meter_Readings-ListView':
            buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Doc_Details-ListView':
            buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs_Out-ListView':
            buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Files-ListView':
            try:
                buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['DD_Files-ListView'].append([index, record, table, operation, None])
            
        if table == 'DD_Docs-ListView':
            buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
            if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                for key in buffer:
                    for j in range(len(buffer[key])):
                        if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                            dict_algoritm_1[user_id].append(buffer[key][j])
                        else:
                            correct_records.append(buffer[key][j][1])
                            
                        use_records.append(buffer[key][j][1])
                        
                        df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
                        
            buffer = {
                'ED_Meter_Readings-ListView': [],
                'DD_Doc_Details-ListView': [],
                'DD_Docs_Out-ListView': [],
                'DD_Files-ListView': [],
                'DD_Docs-ListView': [],
                'Other': []
            }
            
        if table not in (['ED_Meter_Readings-ListView', 'DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_1[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [477]:
# use_records = []
# correct_records = []
# dict_algoritm_1 = {'7d865ee6-9803-427f-81f4-c8f956f6b46b': []}
# algoritm_1('7d865ee6-9803-427f-81f4-c8f956f6b46b')

# df['Use'] = df['C_Record_ID'].isin(use_records)

# display(df.loc[df['F_User'] == '7d865ee6-9803-427f-81f4-c8f956f6b46b', ['C_Record_ID', 'C_Table', 'D_Date_Created', 'Group_ID', 'Use']].sort_values(by=['D_Date_Created'], ascending=True).head(50))

In [478]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_1(user_id)

In [479]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 25165


In [480]:
df.to_excel('outputs/algoritm_1.xlsx')

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [481]:
# dict_algoritm_2 = {}
# for i in range(len(users)):
#     dict_algoritm_2[users[i]] = []

In [482]:
# def algoritm_2(user_id):    
#     items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

#     indexes = list(items.index)
#     tables = list(items['C_Table'])
#     operations = list(items['N_Operation'])
#     json_items = list(items['C_Json'])
#     resords = list(items['C_Record_ID'])

#     buffer = {
#         'ED_Meter_Readings-ListView': [],
#         'DD_Docs_Out-ListView': [],
#         'DD_Files-ListView': [],
#         'DD_Docs-ListView': [],
#         'Other': []
#     }
    
#     is_meter = False
#     is_not_meter = False
    
#     for i in range(len(tables)):
#         table = tables[i]
#         operation = operations[i]
#         json_item = json_items[i]
#         index = indexes[i]
#         record = resords[i]

#         if table == 'ED_Meter_Readings-ListView':
#             is_meter = True
            
#             if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                 
#                 for key in buffer:
#                     for j in range(len(buffer[key])):
#                         if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
#                             dict_algoritm_2[user_id].append(buffer[key][j])
#                         else:
#                             correct_records.append(buffer[key][j][1])
                            
#                         use_records.append(buffer[key][j][1])
                            
#             if is_not_meter == True:   
#                 buffer = {
#                     'ED_Meter_Readings-ListView': [],
#                     'DD_Docs_Out-ListView': [],
#                     'DD_Files-ListView': [],
#                     'DD_Docs-ListView': [],
#                     'Other': []
#                 }
                
#             buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
#             is_not_meter = False
#         else:
#             is_meter = False
            
#         if table == 'DD_Docs_Out-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0:
#             buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Files-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0:
#             try:
#                 buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['DD_Files-ListView'].append([index, record, table, operation, None])
            
#         if table == 'DD_Docs-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0:
#             buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             is_not_meter = True
            
#         if table not in (['ED_Meter_Readings-ListView', 'DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
#             try:
#                 buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['Other'].append([index, record, table, operation, None])
            
#     # дозаписываем данные
#     if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
#         for key in buffer:
#             for j in range(len(buffer[key])):
#                 dict_algoritm_2[user_id].append(buffer[key][j])
#                 use_records.append(buffer[key][j][1])

In [483]:
# for i in range(len(users)):
#     user_id = users[i]
#     algoritm_2(user_id)

In [484]:
# df['Use'] = df['C_Record_ID'].isin(use_records)
# print(f'Обработано записей: {len(use_records)}')

In [485]:
# df.to_excel('outputs/algoritm_2.xlsx')

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Docs-ListView

In [486]:
# dict_algoritm_3 = {}
# for i in range(len(users)):
#     dict_algoritm_3[users[i]] = []

In [487]:
# def algoritm_3(user_id):    
#     items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

#     indexes = list(items.index)
#     tables = list(items['C_Table'])
#     operations = list(items['N_Operation'])
#     json_items = list(items['C_Json'])
#     resords = list(items['C_Record_ID'])

#     buffer = {
#         'ED_Meter_Readings-ListView': [],
#         'DD_Doc_Details-ListView': [],
#         'DD_Docs_Out-ListView': [],
#         'DD_Docs-ListView': [],
#         'Other': []
#     }
    
#     is_meter = False
#     is_not_meter = False
    
#     for i in range(len(tables)):
#         table = tables[i]
#         operation = operations[i]
#         json_item = json_items[i]
#         index = indexes[i]
#         record = resords[i]

#         if table == 'ED_Meter_Readings-ListView':
#             is_meter = True
            
#             if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                
#                 for key in buffer:
#                     for j in range(len(buffer[key])):
#                         if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
#                             dict_algoritm_3[user_id].append(buffer[key][j])
#                         else:
#                             correct_records.append(buffer[key][j][1])
                            
#                         use_records.append(buffer[key][j][1])
                            
#             if is_not_meter == True:   
#                 buffer = {
#                     'ED_Meter_Readings-ListView': [],
#                     'DD_Doc_Details-ListView': [],
#                     'DD_Docs_Out-ListView': [],
#                     'DD_Docs-ListView': [],
#                     'Other': []
#                 }
                
#             buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
#             is_not_meter = False
#         else:
#             is_meter = False
            
#         if table == 'DD_Doc_Details-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0:
#             buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Docs_Out-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0:
#             buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Docs-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0:
#             buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             is_not_meter = True
        
#         if table not in (['ED_Meter_Readings-ListView', 'DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Docs-ListView']):
#             try:
#                 buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['Other'].append([index, record, table, operation, None])
            
#     # дозаписываем данные
#     if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
#         for key in buffer:
#             for j in range(len(buffer[key])):
#                 dict_algoritm_3[user_id].append(buffer[key][j])
#                 use_records.append(buffer[key][j][1])

In [488]:
# for i in range(len(users)):
#     user_id = users[i]
#     algoritm_3(user_id)

In [489]:
# df['Use'] = df['C_Record_ID'].isin(use_records)
# print(f'Обработано записей: {len(use_records)}')

In [490]:
# df.to_excel('outputs/algoritm_3.xlsx')

## Алгоритм

Последовательность DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Docs-ListView

In [491]:
dict_algoritm_4 = {}
for i in range(len(users)):
    dict_algoritm_4[users[i]] = []

In [492]:
def algoritm_4(user_id):    
    items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'DD_Doc_Details-ListView': [],
        'DD_Docs_Out-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'DD_Doc_Details-ListView': 
            buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs_Out-ListView':
            buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs-ListView':
            buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
            if len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                for key in buffer:
                    for j in range(len(buffer[key])):
                        if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                            dict_algoritm_4[user_id].append(buffer[key][j])
                        else:
                            correct_records.append(buffer[key][j][1])

                        use_records.append(buffer[key][j][1])

                        df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
                            
            buffer = {
                'DD_Doc_Details-ListView': [],
                'DD_Docs_Out-ListView': [],
                'DD_Docs-ListView': [],
                'Other': []
            }
        
        if table not in (['DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_4[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [493]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_4(user_id)

In [494]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 28304


In [495]:
df.to_excel('outputs/algoritm_4.xlsx')

## Алгоритм

Последовательность DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [496]:
dict_algoritm_5 = {}
for i in range(len(users)):
    dict_algoritm_5[users[i]] = []

In [497]:
def algoritm_5(user_id):    
    items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'DD_Docs_Out-ListView': [],
        'DD_Files-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'DD_Docs_Out-ListView':
            buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Files-ListView':
            try:
                buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['DD_Files-ListView'].append([index, record, table, operation, None])
            
        if table == 'DD_Docs-ListView':
            buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
            if len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                for key in buffer:
                    for j in range(len(buffer[key])):
                        if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                            dict_algoritm_5[user_id].append(buffer[key][j])
                        else:
                            correct_records.append(buffer[key][j][1])
                            
                        use_records.append(buffer[key][j][1])
                        
                        df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
            buffer = {
                'DD_Docs_Out-ListView': [],
                'DD_Files-ListView': [],
                'DD_Docs-ListView': [],
                'Other': []
            }
        
        if table not in (['DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_5[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [498]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_5(user_id)

In [499]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 28846


In [500]:
df.to_excel('outputs/algoritm_5.xlsx')

In [501]:
df['Group_ID'].nunique()

4611

In [502]:
df['F_Doc_Details'] = ''

In [503]:
df['C_Table'].isin(['ED_Meter_Readings-ListView', 'DD_Doc_Details-ListView']).sum()

9565

## Заполнение DD_Doc_Details

In [504]:
df['Fixed'] = df['Use']
df['F_Doc_Details'] = ''
groups = list(df['Group_ID'].unique())
for groups_id in groups:
    data = df[(df['Group_ID'] == groups_id) & (record['Use'] == True)]

    detail_links = []

    meters_index = data['C_Table'].isin(['ED_Meter_Readings-ListView'])
    meters_data = data[meters_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
    if len(meters_data) > 0:
        for i in meters_data:
            if bool(i['F_Doc_Details']):
                detail_links.append(i['F_Doc_Details']['LINK'])

    
    details_index = data['C_Table'].isin(['DD_Doc_Details-ListView'])
    details_data = data[details_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
    if len(details_data) > 0:
        for i in details_data:
            detail_links.append(i['LINK'])

    found_detail = list(set(detail_links))

    if len(found_detail) > 1:
        df.loc[data.index, 'F_Doc_Details'] = json.dumps(found_detail)
        df.loc[data.index, 'Fixed'] = False

    if len(found_detail) == 1:
        df.loc[data.index, 'F_Doc_Details'] = found_detail[0]
        df.loc[data.index, 'Fixed'] = True

    if len(found_detail) == 0:
        df.loc[data.index, 'F_Doc_Details'] = ''


In [505]:
df.to_excel('outputs/F_Doc_Details.xlsx')

## Объединяем группы DD_Docs_Out
Сперва все у кого DD_Doc_Details одинаковый берем как один акт а потом если у них операция = 0 то обратно оставляем Group_ID, отдельные группы

In [506]:
fixed_groups = df[(df['Fixed'] == True) & (df['F_Doc_Details'] != '')][['Group_ID', 'F_Doc_Details']].groupby(['F_Doc_Details']).agg({'Group_ID': 'first'}).reset_index().rename(columns={'Group_ID': 'Group_ID_Fixed'})

In [507]:
df = df.merge(fixed_groups, left_on='F_Doc_Details', right_on='F_Doc_Details')
df[df['F_Doc_Details'] == 'a95caaa0-a674-4f31-84d4-cbb52f0317f1']

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created,Group_ID,Use,F_Doc_Details,Fixed,Group_ID_Fixed
0,64b18dc9-69d0-4286-887f-7469ebc1b47d,9770f581-b89f-4fb3-94f5-93173bb85ad1,ED_Meter_Readings-ListView,1,"{""LINK"":""50fb47a1-6711-4e65-9232-3700daaa58bd""...",2023-04-09 16:42:47.637,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
1,64b18dc9-69d0-4286-887f-7469ebc1b47d,5f97a8be-b3ad-4982-9191-fbed301b3c69,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 16:42:47.980,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
2,64b18dc9-69d0-4286-887f-7469ebc1b47d,d4c614dd-3114-4326-8ddd-bb9d74382b03,DD_Docs_Out-ListView,0,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:48.627,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
3,64b18dc9-69d0-4286-887f-7469ebc1b47d,9ee02cde-4a6f-4c51-8e3b-b14ddae38bee,DD_Files-ListView,0,"{""LINK"":""e010d6d0-1df5-790d-19fe-b007d7be4207""...",2023-04-09 16:42:49.557,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
4,64b18dc9-69d0-4286-887f-7469ebc1b47d,6829a2cd-007c-45d8-88ca-e143617555c4,DD_Docs-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:50.010,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
5,64b18dc9-69d0-4286-887f-7469ebc1b47d,9c968f25-e25e-4082-b15d-cdb26fa88aee,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 16:57:15.560,a0ec21d9-9722-4e23-9b03-b36ca2f1f252,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
6,64b18dc9-69d0-4286-887f-7469ebc1b47d,a0ec21d9-9722-4e23-9b03-b36ca2f1f252,DD_Docs_Out-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:57:16.460,a0ec21d9-9722-4e23-9b03-b36ca2f1f252,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
7,64b18dc9-69d0-4286-887f-7469ebc1b47d,59e00450-e9f8-45a3-aacb-e95521632320,DD_Docs-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:57:16.960,a0ec21d9-9722-4e23-9b03-b36ca2f1f252,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
8,64b18dc9-69d0-4286-887f-7469ebc1b47d,194df06f-a2a2-42bb-939b-331d0281a3af,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 22:49:15.633,02d08055-5bb7-4c85-926d-87d628637c25,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
9,64b18dc9-69d0-4286-887f-7469ebc1b47d,02d08055-5bb7-4c85-926d-87d628637c25,DD_Docs_Out-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 22:49:16.377,02d08055-5bb7-4c85-926d-87d628637c25,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03


In [508]:
df.to_excel('outputs/Group_Fixed.xlsx')

In [514]:
# берем все записи где надо взять ID из Group_ID, если это операция апдейта то возвращаем значения из Group_ID в Group_ID_Fixed
Group_To_Fix = (df[(df['Group_ID'] != df['Group_ID_Fixed']) & (df['N_Operation'] == 0) & (df['C_Table'] == 'DD_Docs_Out-ListView')])['Group_ID'].unique()


In [517]:
df.loc[df['Group_ID'].isin(Group_To_Fix), 'Group_ID_Fixed'] = df['Group_ID']

In [518]:
df['Group_ID'] = df['Group_ID_Fixed']
df = df.drop(['Group_ID_Fixed'], axis=1)


## Заполняем JSON
Берем Group_ID и F_Doc_Details, заполняем значения

In [519]:
errors = 0

for record_id in df.index:
    record = df.loc[record_id]
    if (record['Use']) & (record['Fixed']):
        if record['C_Table'] == 'DD_Docs_Out-ListView':
            obj = json.loads(record['C_Json'])
            obj['LINK'] = record['Group_ID']
            obj['F_Doc_Details'] = {'LINK': record['F_Doc_Details']}
            df.loc[record_id, 'C_Json'] = json.dumps(obj)
            # df.loc[record_id, 'Fixed'] = True

        try:
            if record['C_Table'] == 'DD_Files-ListView':
                obj = json.loads(record['C_Json'])
                obj['F_Doc_Details'] = {'LINK': record['F_Doc_Details']}
                df.loc[record_id, 'C_Json'] = json.dumps(obj)
                # df.loc[record_id, 'Fixed'] = True
            
        except TypeError as e:
            errors += 1
            df.loc[record_id, 'Fixed'] = False
            # print(record['Group_ID'], record['C_Record_ID'])

In [245]:
df.to_excel('outputs/DD_Files.xlsx')

In [520]:
df[df['C_Record_ID'].isin(correct_records)].sort_values(by=['D_Date_Created'], ascending=True)

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created,Group_ID,Use,F_Doc_Details,Fixed
514,64b18dc9-69d0-4286-887f-7469ebc1b47d,684df633-d275-407b-8630-51d18cd11300,DD_Doc_Details-ListView,1,"{""LINK"":""8c110804-0515-46ac-bc75-e1677d67ea45""...",2023-04-09 18:42:55.083,8829a30f-1e54-4484-8d88-d14a53aaa17f,True,8c110804-0515-46ac-bc75-e1677d67ea45,True
515,64b18dc9-69d0-4286-887f-7469ebc1b47d,8829a30f-1e54-4484-8d88-d14a53aaa17f,DD_Docs_Out-ListView,1,"{""LINK"": ""8829a30f-1e54-4484-8d88-d14a53aaa17f...",2023-04-09 18:42:55.847,8829a30f-1e54-4484-8d88-d14a53aaa17f,True,8c110804-0515-46ac-bc75-e1677d67ea45,True
516,64b18dc9-69d0-4286-887f-7469ebc1b47d,51eeeb47-96e1-4479-9bc7-0d3e99d62b36,DD_Docs-ListView,1,"{""LINK"":""382f7b07-9e6f-42fb-8a81-05025b6227a0""...",2023-04-09 18:42:56.330,8829a30f-1e54-4484-8d88-d14a53aaa17f,True,8c110804-0515-46ac-bc75-e1677d67ea45,True
520,64b18dc9-69d0-4286-887f-7469ebc1b47d,a0c82551-eb25-41cd-9ddf-1993c6de5a2d,DD_Doc_Details-ListView,1,"{""LINK"":""253a53bb-262e-4495-8f82-a93aa36e6f10""...",2023-04-09 18:44:12.183,a23773d3-13eb-450f-a2e4-59ff1570cf4c,True,253a53bb-262e-4495-8f82-a93aa36e6f10,True
521,64b18dc9-69d0-4286-887f-7469ebc1b47d,a23773d3-13eb-450f-a2e4-59ff1570cf4c,DD_Docs_Out-ListView,1,"{""LINK"": ""a23773d3-13eb-450f-a2e4-59ff1570cf4c...",2023-04-09 18:44:13.007,a23773d3-13eb-450f-a2e4-59ff1570cf4c,True,253a53bb-262e-4495-8f82-a93aa36e6f10,True
...,...,...,...,...,...,...,...,...,...,...
28116,f7b6d686-8b6f-4082-af21-056f3fd03375,3c07f448-cbac-45d2-b2e2-0132c4febc00,DD_Docs_Out-ListView,1,"{""LINK"": ""3c07f448-cbac-45d2-b2e2-0132c4febc00...",2023-04-10 23:07:05.917,3c07f448-cbac-45d2-b2e2-0132c4febc00,True,5a7a0ded-87a3-435b-b897-d64d6efd5074,True
28117,f7b6d686-8b6f-4082-af21-056f3fd03375,9618f418-cd54-474b-84ee-3d223fae8847,DD_Docs-ListView,1,"{""LINK"":""287be19f-7654-4d52-9abd-915812d284bd""...",2023-04-10 23:07:06.333,3c07f448-cbac-45d2-b2e2-0132c4febc00,True,5a7a0ded-87a3-435b-b897-d64d6efd5074,True
28118,f7b6d686-8b6f-4082-af21-056f3fd03375,9915576d-b2d2-4694-aad6-a9cbd570bef1,DD_Doc_Details-ListView,1,"{""LINK"":""da7b0754-aa77-4229-8f9c-5054c75ff205""...",2023-04-10 23:13:32.787,a513cc08-a59a-4b05-8a77-1967dec706ac,True,da7b0754-aa77-4229-8f9c-5054c75ff205,True
28119,f7b6d686-8b6f-4082-af21-056f3fd03375,a513cc08-a59a-4b05-8a77-1967dec706ac,DD_Docs_Out-ListView,1,"{""LINK"": ""a513cc08-a59a-4b05-8a77-1967dec706ac...",2023-04-10 23:13:33.687,a513cc08-a59a-4b05-8a77-1967dec706ac,True,da7b0754-aa77-4229-8f9c-5054c75ff205,True


In [522]:
df[(df['Use'] == True) & (df['Fixed'] == True)].rename(columns={'Fixed':'is_fixed'}).to_excel('outputs/Export.xlsx')